In [1]:
import requests
import json
from bs4 import BeautifulSoup
from csv import writer

# Use your own client credentials
client_id1 = 'client_4253061ce6d6a7a86cc877d9c9dfa776'
client_secret1 = 'secret_2e3838d9fe9ced976a3fa974689a9d9d'
client_id2 = 'client_a89a8ef1b3cf662607f452a01c8f27b1'
client_secret2 = 'secret_a4f96d2c0790ad05fe041eed2c314dfa'

client = {}
client['client_id'] = [client_id1, client_id2],
client['client_secret'] = [client_secret1, client_secret2]

def retrive_properties(client_id, client_secret, pageNumber, postCode="", state="VIC", pageSize=200, filename="propertiesInfo.csv"):
  scopes = ['api_listings_read']
  auth_url = 'https://auth.domain.com.au/v1/connect/token'
  url_endpoint = 'https://api.domain.com.au/v1/listings/residential/_search'

  # mock client post request in exchange of a one-time access token
  response = requests.post(auth_url, data = {
                          'client_id':client_id,
                          'client_secret':client_secret,
                          'grant_type':'client_credentials',
                          'scope':scopes,
                          'Content-Type':'text/json'
                          })
  json_res = response.json()
  # take the access token each time (can call a page of size 200)
  access_token=json_res['access_token']
  print("Access:", access_token)
  # authentication for current access token
  auth = {'Authorization':'Bearer ' + access_token}
  url = url_endpoint

  # with the current access token, specify the post search query 
  post_fields = {
        "listingType":"Rent", # only interested in Rents
        "pageSize": pageSize, 
        "pageNumber": pageNumber, # 200 per page
        "propertyTypes":"",
        "minBedrooms":1,
        "minBathrooms":1,
        # "maxPrice": 400,
        "sort": {
            "sortKey": "Default"
        },

        "locations":[
          {
            "state":state,
            "region":"",
            "area":"",
            "suburb":"",
            "postCode":postCode, # will iterate by postcodes
            "includeSurroundingSuburbs":False
          }
        ]
  }

  # get the response of a post search, could either be 200 (success) or some errors
  respond = requests.post(url,headers=auth,json=post_fields)
  content = json.loads(respond.text)
  print("Loaded data: " + str(len(content)))
  # quota error or no relevant data can be found on this page, e.g.
  # {'errors': {'searchParameters.Page': ['Cannot page beyond 1000 records.']},
  # 'message': 'The request is invalid.'}
  if ("errors" in content or len(content) == 0):                                              
    print("Limit notice")
    print(content)
    return 1
  # errors occur when the query type is not from PropertyListing
  elif ("type" in content and content["type"] != "PropertyListing"):
    print("Error accurs")
    print(content)
    return 2

  print(content[0])
  # Get request for each property ID and extract attributes, each 'i' is 
  # a single property schema in json format
  for i in content:
      #get details
      id = i['listing']['id']
      listing_type = i['listing']['listingType']
      r = i['listing']['propertyDetails']
      street_address=r['displayableAddress']
      suburb=r['suburb']
      postcode=r['postcode']
      area=r['area']
      if('buildingArea' in r and 'landArea' in r):
        building_area = r['buildingArea']
        land_area = r['landArea']
      else:
        building_area = 0
        land_area = 0
      # latitude and longitude might not be available in some instances
      if ('latitude' in r and 'longitude' in r):
        latitude = r['latitude']
        longitude = r['longitude']
      else:
        latitude = False
        longitude = False
      
      property_type=r['propertyType']
      bathrooms=r['bathrooms']
      bedrooms=r['bedrooms']

      if(str(i).find('carspaces'))>0:
          carspaces=i['listing']['propertyDetails']['carspaces']
      else:
          carspaces=0

      price=i['listing']['priceDetails']['displayPrice'] #[1:4]

      if(i['listing']['dateListed']):
          time = i['listing']['dateListed']
      else:
          time = 0

      info = [id, listing_type, price, time, property_type, area, building_area, land_area, 
      bedrooms, bathrooms, carspaces, street_address, suburb, postcode, latitude, longitude]

      # write the relevant info of current property one row at a time
      with open(filename, 'a', newline='') as g:
          thewriter = writer(g)
          thewriter.writerow(info)

  return 0

# retrive_properties(client_id, client_secret, 1)

### Inialise the csv output file with headers

In [2]:
# Write column headers
headers = ['property_id', 'listing_type', 'price', 'time_listed', 'property_type', 'area', 'building_area', 'land_area', 
'bedrooms', 'bathrooms', 'carspaces', 'street_address', 'suburb', 'postcode', 'latitude', 'longitude']
output_csv = open('propertiesInfo.csv', 'w')
# create the csv writer object
csv_writer = writer(output_csv)
# first write the headers
csv_writer.writerow(headers)

output_csv.close()

Rrtrive rental data

In [3]:
# Retrive rental data by postcode
# postcode 3000~4000
count = 0

for i in range(3000, 4000):
    print("Retriving data from postcode: " + str(i))
    if(count >= len(client['client_id']) or count >= len(client['client_secret'])):
        break
    client_id = client['client_id'][count]
    client_secret = client['client_secret'][count]

    for p in range(1, 6):
        num = retrive_properties(client_id, client_secret, p, str(i))
        if num == 1:
            break
        elif num == 2:
            count += 1
            retrive_properties(client_id, client_secret, p, str(i))

# postcode 8000~9000
for i in range(8000, 9000):
    print("Retriving data from postcode: " + str(i))
    if(count >= len(client['client_id']) or count >= len(client['client_secret'])):
        break
    client_id = client['client_id'][count]
    client_secret = client['client_secret'][count]
    for p in range(1, 6):
        num = retrive_properties(client_id, client_secret, p, str(i))
        if num == 1:
            break
        elif num == 2:
            count += 1
            retrive_properties(client_id, client_secret, p, str(i))


Retriving data from postcode: 3000
Access: abe0d122c762bd8f8c10536b980dcd89
Loaded data: 200
{'type': 'PropertyListing', 'listing': {'listingType': 'Rent', 'id': 16064379, 'advertiser': {'type': 'Agency', 'id': 345, 'name': 'Dingle Partners', 'logoUrl': 'https://images.domain.com.au/img/Agencys/345/logo_345.png', 'preferredColourHex': '#fff', 'bannerUrl': 'https://images.domain.com.au/img/Agencys/345/banner_345.png', 'contacts': [{'name': 'Justine Muscat'}, {'name': 'Kai Lupton'}]}, 'priceDetails': {'displayPrice': '$370 per week'}, 'media': [{'category': 'Image', 'url': 'https://bucket-api.domain.com.au/v1/bucket/image/16064379_1_1_220830_052022-w4000-h2670'}, {'category': 'Image', 'url': 'https://bucket-api.domain.com.au/v1/bucket/image/16064379_2_1_220830_052022-w4000-h2670'}, {'category': 'Image', 'url': 'https://bucket-api.domain.com.au/v1/bucket/image/16064379_3_1_220830_052022-w4000-h2670'}, {'category': 'Image', 'url': 'https://bucket-api.domain.com.au/v1/bucket/image/16064379_